# Used Cars

Nå skal vi forsøke å anvende maskinlæringsmodeller på bruktbilmarkedet. Used cars inneholder informasjon om ca. 150 bruktbiler som har blitt solgt, og til hvilken pris. I denne oppgaven får du prøve å predikere/forutsi på hvilken girkasse den solgte bilen hadde, og etterpå gjette på hvilken pris bilen ble solgt for.

## Forberedelser

Importer nødvendige biblioteker

In [0]:
# Biblioteket som vi bruker for å lese og behandle datasettet som kommer fra CSV.
import pandas as pd

# I supervised learning er hver input koblet til en fasit, også kalt "label".
# LabelEncoder brukes for å gjøre fasiten enklere å behandle.
from sklearn.preprocessing import LabelEncoder

# Til slutt må vi importere maskinlæringsmodellene som vi bruker for å trene.
from sklearn import model_selection, metrics, svm

Bruk pandas og importer datasettet som ligger https://github.com/djick/ML101/blob/master/data/usedcars.csv

In [0]:
!wget -q https://raw.githubusercontent.com/djick/ML101/master/data/usedcars.csv -O usedcars.csv
usedcars = pd.read_csv("usedcars.csv")

Undersøk hva som faktisk ligger i datasettet, hva som skal gå inn i maskinlæringsmodellen, og modellen skal prøve å forutsi.

In [0]:
print("De første radene i CSV-filen:")
print(usedcars.head(10))

Her får vi se alle featurene som vi har å jobbe med, og er det jo noen muligheter for hva man kan prøve å forutsi. Hvilken som helst av disse kolonnene kan være labelen, altså det man maskinlæringsmodellen skal prøve å gjette.

Videre, gjør litt statistikk og analyse av hvordan de numeriske verdiene er distribuert hvis du vil.

In [0]:
print("Statistiske verdier for datasettet:")
print(usedcars.describe())

print("\nKlassedistrubisjon: ")
print()
print(usedcars.groupby("transmission").size())
print()
print(usedcars.groupby("model").size())
print()
print(usedcars.groupby("color").size())

import matplotlib.pyplot as plt
print("\nPris:")
plt.hist(usedcars["price"])
plt.show

Da maskinlæringsmodellene som vi bruker liker numeriske verdier bedre enn ord må vi endre på noe av datasettet. 






In [0]:
encoder = LabelEncoder()
# Tilpasser encoderen til featuren "model"
encoder.fit(usedcars["model"])
# Endrer den faktiske verdien i datasettet. Modellen "SEL" blir f.eks. til klasse "1".
usedcars["model"] = encoder.fit_transform(usedcars["model"])

# Resetter encoderen
encoder = LabelEncoder()
# Gjør det samme for "color"
encoder.fit(usedcars["color"])
usedcars["color"] = encoder.fit_transform(usedcars["color"])


Vi har to kategorier med potensielle labels som man kan bruke til enten

1. classification: Labelsene puttes i hver sin boks eller klasse. F.eks. verdiene i kolonnen "color". Fargen gul har ikke noen sammenheng med fargen hvit, derav klassene. Totalt så har vi `model`, `color`, `transmission` i vårt datasett.
2. eller regression: Labelsene finnes på samme dimensjon. Pris er et bra eksempel. I vårt datasett har vi `year`, `price` og `mileage`.

Sjekk nå hvordan datasettet ser ut. Du *kan* også sjekke hvordan distribusjonen av featurene er nå, men det skal være det samme som før fordi det eneste du gjorde var å oversette featurene.

In [0]:
# TODO print ut noen rader og sjekk.
print(usedcars.head(10))

Separer features fra targets. Targets er hva du har lyst å prøve å forutsi.

In [0]:
verdier = usedcars[["year", "model", "price", "mileage", "color"]]
klasser = usedcars["transmission"]

Nå er tiden inne for å dele datasettet opp i trening og test.

In [0]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(verdier, klasser, test_size=0.33, random_state=1)

`X` er featurene og `y` er labelsene. `test_size=0.33` betyr at du tar vare på 33 % av hele datasettet til å validere hvor bra modellen ble trent. `random_state` settes for å shuffle til alle bilene; da unngår man at test- og treningsgruppene blir skjevt fordelt.

## Tren modellen
Forarbeidet er gjort. Vi må velge hvilken maskinlæringsmodell vi vil gå for, og deretter trene modellen. Det finnes ganske mange modeller å gå for, alt etter hva `target` er. Det vi vet helt sikkert er at vi skal gå for modeller innenfor supervised learning fordi vi har fasiten, og at vi skal gå for classification (ikke regression) fordi vi skal prøve å forutsi transmission-type. Nå er google din venn, og Scikit learn sine websider har bra med dokumentasjon og tutorials.

Et raskt websøk gir https://medium.com/cracking-the-data-science-interview/a-tour-of-the-top-10-algorithms-for-machine-learning-newbies-7228aa8ef541

En oversikt over hvilke modeller Scikit learn har å tilby: https://scikit-learn.org/stable/supervised_learning.html

In [0]:
## Lag en klassifiseringsmodell
print("Modell:")
# Benytter SVM - Support Vector Machine
classifier = svm.SVC()
print(classifier)
print(60 * "=")

Tren modellen!

In [0]:
classifier.fit(X_train, y_train)

## Test modellen
De 33 % av datasettet vi sparte i stad får vi nå bruk for.

In [0]:
# Prediker (test)
prediksjoner = classifier.predict(X_test)

`prediksjoner` bør nå være så lik som mulig `y_test`.

In [0]:
print("Resultater:")
print(metrics.classification_report(y_test, prediksjoner, target_names=usedcars["transmission"].unique()))
print(60 * "=")

print("Confusion matrix:")
print(metrics.confusion_matrix(y_test, prediksjoner))
print(60 * "=")

Utfør noen stikkprøver og sjekk hvor ofte modellen treffer riktig.

In [0]:
print("Forutsigelser:")
print(y_test.head(10))
print("Fasit:")
print(prediksjoner[0:10])

Hvor bra ble det? Hvorfor ble resultatet slik det ble?

Finnes det andre modeller som gir bedre resultat enn `SVM`?

# Ny modell – pris

Det som kanskje er mer artig er å gjette prisen på en bruktbil. Gjenbruk mye av det ovenfor, men i stedet for å velge `transmission` som target skal du nå velge `price`. Her vil det ikke holde med `SVM` da den brukes til klassifisering. Nå må du se etter en modell som kan utføre regression.

In [0]:
# TODO importer en modell fra sklearn som kan utføre regression
from sklearn.linear_model import LinearRegression

encoder = LabelEncoder()
encoder.fit(usedcars["transmission"])
usedcars["transmission"] = encoder.fit_transform(usedcars["transmission"])

# TODO velg nytt target, dvs. price
verdier = usedcars[["year", "model", "transmission", "mileage", "color"]]
klasser = usedcars["price"]

# TODO del opp i trening og test
X_train, X_test, y_train, y_test = model_selection.train_test_split(verdier, klasser, test_size=0.33, random_state=1)

# TODO opprett en instans av modellen du valgte å importere
classifier = LinearRegression()
print(classifier)
print(60 * "=")

# TODO tren
classifier.fit(X_train, y_train)



In [0]:
print(X_test.head())
print(X_train.head())
print()


# TODO Prediker/gjett!
prediksjoner = classifier.predict(X_test)

print("Fasit:")
print(y_test.head(5))
print("Prediksjoner:")
print(prediksjoner[0:5])

Hvor liten andel trening i forhold til test kan du la modellen trene på, og fortsatt få en modell som gjør det OK?